In [0]:
dbutils.widgets.removeAll()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [0]:
dbutils.widgets.text("catalogo", "catalog_jptq")
dbutils.widgets.text("esquema_source", "bronze_jptq")
dbutils.widgets.text("esquema_target", "golden_jptq")
dbutils.widgets.text("tabla", "TBL_FCT_PELICULA")
dbutils.widgets.text("storageLocation", "abfss://unity-catalog-jptq@adlsjptq0126.dfs.core.windows.net")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_target = dbutils.widgets.get("esquema_target")
tabla = dbutils.widgets.get("tabla")
storageLocation = dbutils.widgets.get("storageLocation")

In [0]:
def fn_create_table_fct_pelicula(catalogo: str, esquema: str, tabla: str):
   if spark.catalog.tableExists(f"{catalogo}.{esquema_target}.{tabla}"):
      print(f"La tabla ya existe: {catalogo}.{esquema_target}.{tabla}")
   else:
      spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalogo}.{esquema_target}.{tabla} (
      SK_DIM_PELICULA INT,
      SK_DIM_DIRECTOR INT,
      SK_DIM_GENERO INT,
      SK_DIM_IDIOMA INT,
      VAL_PUNTUACION_USUARIO DOUBLE, 
      VAL_DURACION_PELICULA_HORA DOUBLE,
      VAL_DURACION_PELICULA_MINUTOS DOUBLE,
      VAL_NUMERO_VOTOS DOUBLE,
      VAL_PRESUPUESTO_USD DOUBLE,
      VAL_GANANCIAS_USD DOUBLE,
      FEC_CARGA DATE
      )
      USING DELTA
      LOCATION '{storageLocation}/golden-jptq/{tabla}'""")
      print(f"Tabla creada correctamente: {catalogo}.{esquema_target}.{tabla}")

In [0]:
    fn_create_table_fct_pelicula(catalogo,esquema_target,tabla)

In [0]:
def fn_insert_table_fct_pelicula(catalogo: str, esquema: str, tabla: str):
    spark.sql(f"""INSERT INTO {catalogo}.{esquema_target}.{tabla} (
    SK_DIM_PELICULA,
    SK_DIM_DIRECTOR,
    SK_DIM_GENERO,
    SK_DIM_IDIOMA,
    VAL_PUNTUACION_USUARIO, 
    VAL_DURACION_PELICULA_HORA,
    VAL_DURACION_PELICULA_MINUTOS,
    VAL_NUMERO_VOTOS,
    VAL_PRESUPUESTO_USD,
    VAL_GANANCIAS_USD,
    FEC_CARGA
    )
    SELECT
    DIM_PELICULA.SK_DIM_PELICULA,
    DIM_DIRECTOR.SK_DIM_DIRECTOR,
    DIM_GENERO.SK_DIM_GENERO,
    DIM_IDIOMA.SK_DIM_IDIOMA,
    P.VAL_PUNTUACION_USUARIO, 
    P.VAL_DURACION_PELICULA_HORA,
    P.VAL_DURACION_PELICULA_MINUTOS,
    P.VAL_NUMERO_VOTOS,
    DP.VAL_PRESUPUESTO_USD,
    DP.VAL_GANANCIAS_USD,
    CURRENT_DATE() AS FEC_CARGA
    FROM {catalogo}.{esquema_source}.TBL_PELICULAS P
    LEFT JOIN {catalogo}.{esquema_source}.TBL_DETALLE_PELICULAS DP
    ON P.ID_PELICULA = DP.ID_PELICULA
    LEFT JOIN {catalogo}.{esquema_target}.TBL_DIM_PELICULA_T1 DIM_PELICULA
    ON P.ID_PELICULA = DIM_PELICULA.COD_PELICULA
    LEFT JOIN {catalogo}.{esquema_target}.TBL_DIM_DIRECTOR_PELICULA_T2 DIM_DIRECTOR
    ON CONCAT(
        SUBSTRING(SPLIT(DP.DES_DIRECTOR,' ')[0],1,3),
        SUBSTRING(ELEMENT_AT(SPLIT(DP.DES_DIRECTOR,' '),-1),1,3)
        ) = DIM_DIRECTOR.COD_DIRECTOR_PELICULA
    LEFT JOIN {catalogo}.{esquema_target}.TBL_DIM_GENERO_PELICULA_T0 DIM_GENERO
    ON UPPER(SUBSTRING(P.DES_GENERO_PELICULA, 1, 3)) = DIM_GENERO.COD_GENERO_PELICULA
    LEFT JOIN {catalogo}.{esquema_target}.TBL_DIM_IDIOMA_PELICULA_T0 DIM_IDIOMA
    ON UPPER(P.DES_IDIOMA_PELICULA) = DIM_IDIOMA.COD_IDIOMA_PELICULA
    """)
    print(f"Insersion Correcta en {catalogo}.{esquema_target}.{tabla}")

In [0]:
fn_insert_table_fct_pelicula(catalogo,esquema_target,tabla)

In [0]:
%sql
SELECT
*
FROM
catalog_jptq.golden_jptq.tbl_fct_pelicula